In [4]:
from coddiwomple.openmm.integrators import OMMLI
from coddiwomple.openmm.propagators import OMMBIP
from coddiwomple.openmm.reporters import OpenMMReporter
from openmmtools.states import ThermodynamicState, SamplerState
from simtk import unit
from copy import deepcopy
import os
import sys
import numpy as np
import pickle
import tqdm
import mdtraj.utils as mdtrajutils
import mdtraj as md
import torch
import torchani
from openmmtools.constants import kB

from openmmtools import cache, utils
from perses.dispersed.utils import check_platform, configure_platform
cache.global_context_cache.platform = configure_platform(utils.get_fastest_platform().getName())
atomic_num_to_symbol_dict = {1: 'H', 6: 'C', 7: 'N', 8: 'O'}
mass_dict_in_daltons = {'H': 1.0, 'C': 12.0, 'N': 14.0, 'O': 16.0}

conducting subsequent work with the following platform: CPU


In [5]:
class ANI1_force_and_energy(object):
    # some class attributes
    mass_unit = unit.dalton
    distance_unit = unit.nanometers
    time_unit = unit.femtoseconds
    energy_unit = unit.kilojoules_per_mole
    ani_distance_unit = unit.angstroms
    hartree_to_kJ_per_mole = 2625.499638
    ani_energy_unit = hartree_to_kJ_per_mole * energy_unit
    nm_to_angstroms = 10.
    angstroms_to_nm = 1e-1

    def __init__(self,
                 model,
                 atoms,
                 platform='cpu',
                 temperature=300 * unit.kelvin
                 ):

        """
        Performs energy and force calculations.
        Slightly modified code from:
            https://gist.github.com/wiederm/7ac5c29e5a0dea9d17ef16dda93fe02d#file-reweighting-py-L42; thanks, Marcus

        Parameters
        ----------
        model: torchani.models object
            model from which to compute energies and forces
        atoms: str
            a string of atoms in the indexed order
        platform : str, default 'cpu',
            platform on which to initialize the model device
        temperature : float * unit.kelvin, default 300 * unit.kelvin
            temperature
        """
        self.model = model
        self.atoms = atoms

        self.platform = platform
        self.device = torch.device(self.platform)
        if self.platform == 'cpu':
            torch.set_num_threads(2)
        else:
            raise Exception(f"we don't support gpu just yet")

        self.species = self.model.species_to_tensor(atoms).to(self.device).unsqueeze(0)
        self.temperature = temperature
        self.beta = 1.0 / (kB * temperature)

        self.W_shads = []
        self.W = []

    def minimize(self,
                 coords: unit.quantity.Quantity,
                 maxiter: int = 1000):
        """
        Minimizes the molecule. Note, we usually don't want to do this given an input structure since they are already distributed i.i.d.

        Parameters
        ----------
        coords:simtk.unit.quantity.Quantity
        maxiter: int
            Maximum number of minimization steps performed.

        Returns
        -------
        coords:simtk.unit.quantity.Quantity
        """

        assert (type(coords) == unit.quantity.Quantity)

        x = coords.value_in_unit(unit.angstrom)
        self.memory_of_energy = []
        print("Begin minimizing...")
        f = optimize.minimize(self._target_energy_function, x, method='BFGS',
                              jac=True, options={'maxiter': maxiter, 'disp': True})

        _logger.critical(f"Minimization status: {f.success}")
        memory_of_energy = copy.deepcopy(self.memory_of_energy)
        self.memory_of_energy = []

        return f.x.reshape(-1, 3) * unit.angstrom, memory_of_energy

    def calculate_force(self,
                        x: unit.quantity.Quantity) -> (unit.quantity.Quantity, unit.quantity.Quantity):
        """
        Given a coordinate set the forces with respect to the coordinates are calculated.

        Parameters
        ----------
        x : array of floats, unit'd (distance unit)
            initial configuration

        Returns
        -------
        F : float, unit'd
        E : float, unit'd
        """
        #assert (type(x) == unit.quantity.Quantity)

        coordinates = torch.tensor([x.value_in_unit(unit.angstroms)],
                                   requires_grad=True, device=self.device, dtype=torch.float32)

        energy_in_hartree = self._calculate_energy(coordinates)

        # derivative of E (in kJ/mol) w.r.t. coordinates (in nm)
        derivative = torch.autograd.grad((energy_in_hartree).sum(), coordinates)[0]

        if self.platform == 'cpu':
            F = -1 * derivative[0].numpy()
        elif self.platform == 'cuda':
            F = - np.array(derivative.cpu())[0]
        else:
            raise RuntimeError('Platform needs to be specified. Either CPU or CUDA.')

        return (F * self.hartree_to_kJ_per_mole * (unit.kilojoule_per_mole / unit.angstrom),
                energy_in_hartree.item() * self.hartree_to_kJ_per_mole * unit.kilojoule_per_mole)

    def _calculate_energy(self, coordinates: torch.tensor):
        """
        Helpter function to return energies as tensor.
        Given a coordinate set the energy is calculated.

        Parameters
        ----------
        coordinates : torch.tensor
            coordinates in angstroms without units attached

        Returns
        -------
        energy_in_hartree : torch.tensor

        """

        # stddev_in_hartree = torch.tensor(0.0,device = self.device, dtype=torch.float64)
        energy_in_hartree = self.model((self.species, coordinates)).energies

        return energy_in_hartree

    def _target_energy_function(self, x) -> (float, np.array):
        """
        Given a coordinate set (x) the energy is calculated in kJ/mol.

        Parameters
        ----------
        x : array of floats, unit'd (distance unit)
            initial configuration

        Returns
        -------
        E : float, unitless (in kJ/mol)
        F_flat : (len(x) * 3, )-shaped np.array, unitless (in kJ/mol / unit.angstrom)
        """
        x = x.reshape(-1, 3) * unit.angstrom
        F, E = self.calculate_force(x)
        F_flat = -np.array(F.value_in_unit(unit.kilojoule_per_mole / unit.angstrom).flatten(), dtype=np.float64)
        self.memory_of_energy.append(E)
        return E.value_in_unit(unit.kilojoule_per_mole), F_flat

    def calculate_energy(self, x: unit.Quantity):
        """
        Given a coordinate set (x) the energy is calculated in kJ/mol.
        Parameters
        ----------
        x : array of floats, unit'd (angstroms)
            initial configuration

        Returns
        -------
        energy : unit.quantity.Quantity
            energy in kJ/mol
        """

        #assert (type(x) == unit.quantity.Quantity)
        coordinates = torch.tensor([x.value_in_unit(unit.angstroms)],
                                   requires_grad=True, device=self.device, dtype=torch.float32)

        energy_in_hartrees = self._calculate_energy(coordinates)
        energy = energy_in_hartrees.item() * self.hartree_to_kJ_per_mole * unit.kilojoule_per_mole
        return energy



In [6]:
class Integrator(OMMLI):
    def __init__(self,
                 temperature=300.0 * unit.kelvin,
                 collision_rate=1.0 / unit.picoseconds,
                 timestep=1.0 * unit.femtoseconds,
                 splitting="V R O R F",
                 constraint_tolerance=1e-6,
                 **kwargs):
        """Create a Langevin integrator with the prescribed operator splitting.

        arguments
            splitting : string, default: "V R O R"
                Sequence of "R", "V", "O" (and optionally "{", "}", "V0", "V1", ...) substeps to be executed each timestep.
                Forces are only used in V-step. Handle multiple force groups by appending the force group index
                to V-steps, e.g. "V0" will only use forces from force group 0. "V" will perform a step using all forces.
            temperature : np.unit.Quantity compatible with kelvin, default: 300.0*unit.kelvin
               Fictitious "bath" temperature
            collision_rate : np.unit.Quantity compatible with 1/picoseconds, default: 1.0/unit.picoseconds
               Collision rate
            timestep : np.unit.Quantity compatible with femtoseconds, default: 1.0*unit.femtoseconds
               Integration timestep
            constraint_tolerance : float, default: 1.0e-8
                Tolerance for constraint solver
        """
        #just super our previous method
        super().__init__(temperature,
                         collision_rate,
                         timestep,
                         splitting,
                         constraint_tolerance,
                         **kwargs)
        

    def _add_V_step(self, force_group="0"):
        """Deterministic velocity update, using only forces from force-group fg.

        arguments
            force_group : str, optional, default="0"
               Force group to use for this step
        """
        self.addComputeSum("old_ke", self._kinetic_energy)

        # update velocities
        if self._mts:
            self.addComputePerDof("v", "v + ((dt / {}) * moddi / m)".format(self._force_group_nV[force_group], force_group))
        else:
            self.addComputePerDof("v", "v + (dt / {}) * moddi / m".format(self._force_group_nV["0"]))

        self.addConstrainVelocities()


        self.addComputeSum("new_ke", self._kinetic_energy)
        self.addComputeGlobal("shadow_work", "shadow_work + (new_ke - old_ke)")
    
    def _add_F_step(self):
        """
        add an moddi update step
        """
        self.addComputePerDof('moddi', 'f')
        
    
    def _add_variables(self):
        super()._add_variables()
        self.addPerDofVariable('moddi', 0)
    
    def _add_integrator_steps(self):
        """
        Add the steps to the integrator--this can be overridden to place steps around the integration.
        """
        super()._add_integrator_steps()
        #self.addUpdateContextState()
    
    @property
    def _step_dispatch_table(self):
        dispatch_table = super()._step_dispatch_table
        dispatch_table['F'] = (self._add_F_step, False) #add a moddi variable
        return dispatch_table
        
    
class Propagator(OMMBIP):
    def __init__(self,
                 openmm_pdf_state,
                 openmm_pdf_state_subset,
                 subset_indices_map,
                 integrator,
                 ani_handler,
                 context_cache=None,
                 reassign_velocities=True,
                 n_restart_attempts=0,
                 reporter = None,
                 **kwargs):
        """
        arguments
            openmm_pdf_state : openmmtools.states.ThermodynamicState
                the pdf state of the propagator
            openmm_pdf_state_subset : openmmtools.states.ThermodynamicState
                the pdf state of the atom subset
            subset_indices_map : dict
                dict of {openmm_pdf_state atom_index : openmm_pdf_state_subset atom index}
            integrator : openmm.Integrator
                integrator of dynamics
            ani_handler : ANI1_force_and_energy
                handler for ani forces and potential energy
            context_cache : openmmtools.cache.ContextCache, optional
                The ContextCache to use for Context creation. If None, the global cache
                openmmtools.cache.global_context_cache is used (default is None).
            reassign_velocities : bool, optional
                If True, the velocities will be reassigned from the Maxwell-Boltzmann
                distribution at the beginning of the move (default is False).
            n_restart_attempts : int, optional
                When greater than 0, if after the integration there are NaNs in energies,
                the move will restart. When the integrator has a random component, this
                may help recovering. On the last attempt, the ``Context`` is
                re-initialized in a slower process, but better than the simulation
                crashing. An IntegratorMoveError is raised after the given number of
                attempts if there are still NaNs. 
            reporter : coddiwomple.openmm.reporter.OpenMMReporter, default None
                a reporter object to write trajectories
        """
        super().__init__(openmm_pdf_state,
                 integrator,
                 context_cache,
                 reassign_velocities,
                 n_restart_attempts)
        #create a pdf state for the subset indices (usually a vacuum system)
        self.pdf_state_subset = openmm_pdf_state_subset
        assert self.pdf_state_subset.temperature == self.pdf_state.temperature, f"the temperatures of the pdf states do not match"
        
        #create a dictionary for subset indices
        self._subset_indices_map = subset_indices_map
        
        #create an ani handler attribute that can be referenced
        self.ani_handler = ani_handler
        
        #create a context for the subset atoms that can be referenced
        self.context_subset, _ = cache.global_context_cache.get_context(self.pdf_state_subset)
        
        #create a reporter for the accumulated works
        self._state_works = {}
        self._state_works_counter = 0
        
        #create a reporter
        self._write_trajectory = False if reporter is None else True
        self.reporter=reporter
        if self._write_trajectory:
            from coddiwomple.particles import Particle
            self.particle = Particle(0)
        else:
            self.particle = None
        
    def _before_integration(self, *args, **kwargs):
        particle_state = args[0] #define the particle state
        n_iterations = args[1] #define the number of iterations
        
        self._current_state_works = [] #define an interim (auxiliary) list that will track the thermodynamic work of the current application
        self._current_state_works.append(0.0) #the first incremental work is always 0 since the importance function is identical to the first target distribution (i.e. fully interacting MM)
        
        self._iteration = 0.0 #define the first iteration as 0
        self._n_iterations = n_iterations #the number of iterations in the protocol is equal to the number of steps in the application
        
        #update the particle state and the particle state subset
        particle_state.update_from_context(self.context, ignore_velocities=True) #update the particle state from the context
        self.particle_state_subset = SamplerState(positions = particle_state.positions[list(self._subset_indices_map.keys())]) #create a particle state from the subset context
        self.particle_state_subset.apply_to_context(self.context_subset, ignore_velocities=True) #apply the subset particle state to its context
        self.particle_state_subset.update_from_context(self.context_subset, ignore_velocities=True) #update the subset particle state from its context to updated the potential energy
           
        #get the reduced potential
        reduced_potential = self._compute_hybrid_potential(_lambda = self._iteration / self._n_iterations, particle_state = particle_state)
        perturbed_reduced_potential = self._compute_hybrid_potential(_lambda = (self._iteration + 1.0) / self._n_iterations, particle_state = particle_state)
        self._current_state_works.append(self._current_state_works[-1] + (perturbed_reduced_potential - reduced_potential))
        #print(f"reduced_potential: {reduced_potential}; perturbed_potential: {perturbed_reduced_potential}")
        
        #make a new force object
        mm_force_matrix = self._compute_hybrid_forces(_lambda = (self._iteration + 1.0) / self._n_iterations, particle_state = particle_state).value_in_unit_system(unit.md_unit_system)
        self.integrator.setPerDofVariableByName('moddi', mm_force_matrix) 
        
        #report
        if self._write_trajectory:
            self.particle.update_state(particle_state)
            self.reporter.record([self.particle])
    
    def _during_integration(self, *args, **kwargs):
        particle_state = args[0]
        self._iteration += 1.0
        
        
        #update the particle state and the particle state subset
        particle_state.update_from_context(self.context, ignore_velocities=True) #update the particle state from the context
        self.particle_state_subset.positions = particle_state.positions[list(self._subset_indices_map.keys())] #update the particle subset positions appropriately
        self.particle_state_subset.apply_to_context(self.context_subset, ignore_velocities=True) #apply the subset particle state to its context
        self.particle_state_subset.update_from_context(self.context_subset, ignore_velocities=True) #update the subset particle state from its context to updated the potential energy
        
        #get the reduced potential
        if self._iteration < self._n_iterations:
            reduced_potential = self._compute_hybrid_potential(_lambda = self._iteration / self._n_iterations, particle_state = particle_state)
            perturbed_reduced_potential = self._compute_hybrid_potential(_lambda = (self._iteration + 1.0) / self._n_iterations, particle_state = particle_state)
            self._current_state_works.append(self._current_state_works[-1] + (perturbed_reduced_potential - reduced_potential))
            
            #and create a new modified force
            mm_force_matrix = self._compute_hybrid_forces(_lambda = (self._iteration + 1.0) / self._n_iterations, particle_state = particle_state).value_in_unit_system(unit.md_unit_system)
            self.integrator.setPerDofVariableByName('moddi', mm_force_matrix) 
        else:
            #we are done
            pass
        
        if self._write_trajectory:
            self.particle.update_state(particle_state)
            if self._iteration == self._n_iterations:
                self.reporter.record([self.particle], save_to_disk=True)
            else:
                self.reporter.record([self.particle], save_to_disk=False)
            
            
        
    def _after_integration(self, *args, **kwargs):
        self._state_works[self._state_works_counter] = deepcopy(self._current_state_works)
        self._state_works_counter += 1
        
        if self._write_trajectory:
            self.reporter.reset()
        

    def _compute_hybrid_potential(self,_lambda, particle_state):
        """
        function to compute the hybrid reduced potential defined as follows:
        U(x_rec, x_lig) = u_mm,rec(x_rec) - lambda*u_mm,lig(x_lig) + lambda*u_ani,lig(x_lig)
        """
        reduced_potential = (self.pdf_state.reduced_potential(particle_state)
                             - _lambda * self.pdf_state_subset.reduced_potential(self.particle_state_subset)
                             + _lambda * self.ani_handler.calculate_energy(self.particle_state_subset.positions) * self.pdf_state.beta)
        return reduced_potential
    
    def _compute_hybrid_forces(self, _lambda, particle_state):
        """
        function to compute a hybrid force matrix of shape num_particles x 3
        in the spirit of the _compute_hybrid_potential, we compute the forces in the following way
            F(x_rec, x_lig) = F_mm(x_rec, x_lig) - lambda * F_mm(x_lig) + lambda * F_ani(x_lig)
        """
        # get the complex mm forces
        state = self.context.getState(getForces=True)
        mm_force_matrix = state.getForces(asNumpy=True)

        # get the ligand mm forces
        subset_state = self.context_subset.getState(getForces=True)
        mm_force_matrix_subset = subset_state.getForces(asNumpy=True)

        # get the ligand ani forces
        coords = self.particle_state_subset.positions
        subset_ani_force_matrix, energie = self.ani_handler.calculate_force(coords)

        # now combine the ligand forces
        subset_force_matrix = _lambda * (subset_ani_force_matrix - mm_force_matrix_subset)

        # and append to the complex forces...
        mm_force_matrix[list(self._subset_indices_map.keys()), :] += subset_force_matrix

        return mm_force_matrix
    
    @property
    def state_works(self):
        return self._state_works      

In [7]:
def annealed_importance_sampling(system,
                                 system_subset,
                                 subset_indices_map,
                                 endstate_cache_filename,
                                 directory_name,
                                 trajectory_prefix,
                                 md_topology,
                                 number_of_applications,
                                 steps_per_application,
                                 integrator_kwargs = {'temperature': 300.0 * unit.kelvin,
                                                      'collision_rate': 1.0 / unit.picoseconds,
                                                      'timestep': 1.0 * unit.femtoseconds,
                                                      'splitting': "V R O R F",
                                                      'constraint_tolerance': 1e-6,
                                                      'pressure': 1.0 * unit.atmosphere},
                                 save_indices = None
                                ):
    """
    conduct annealed importance sampling in the openmm regime

    arguments
        system : openmm.System
            system
        system_subset : openmm.System
            subset system
        subset_indices_map : dict
            dict of {openmm_pdf_state atom_index : openmm_pdf_state_subset atom index}
        endstate_cache_filename : str
            path to the endstate cache pdb
        directory_name : str
            directory that will be written to
        trajectory_prefix : str
            .pdb prefix
        md_topology : mdtraj.Topology
            topology that will write the trajectory
        save_indices : list
            list of indices that will be saved 
        number_of_applications : int
            number of applications of the propagator
        steps_per_application : int
            number of integration steps per application
        integrator_kwargs : dict, see default
            kwargs to pass to OMMLIAIS integrator
        save_indices : list(int)
            list of indices of md_topology atoms to save to disk
    """
    #load the endstate cache
    traj = md.Trajectory.load(endstate_cache_filename)
    num_frames = traj.n_frames
    
    #make a handle object for ANI
    species_str = ''.join([atom.element.symbol for atom in md_topology.subset(list(subset_indices_map.keys())).atoms])
    print(f"species string: {species_str}")
    ani_handler = ANI1_force_and_energy(model = torchani.models.ANI1ccx(),
                                                 atoms=species_str,
                                                 platform='cpu',
                                                 temperature=integrator_kwargs['temperature'])

    #make thermostates
    pdf_state = ThermodynamicState(system = system, temperature = integrator_kwargs['temperature'], pressure= integrator_kwargs['pressure'])
    pdf_state_subset = ThermodynamicState(system = system_subset, temperature = integrator_kwargs['temperature'], pressure = integrator_kwargs['pressure'])
    
    #make a reporter
    reporter = OpenMMReporter(directory_name, trajectory_prefix, md_topology, subset_indices = save_indices)
    
    
    #make an integrator
    integrator = Integrator(**integrator_kwargs)
    
    #make a propagator
    propagator = Propagator(openmm_pdf_state = pdf_state,
                 openmm_pdf_state_subset = pdf_state_subset,
                 subset_indices_map = subset_indices_map,
                 integrator = integrator,
                 ani_handler = ani_handler,
                 context_cache=None,
                 reassign_velocities=True,
                 n_restart_attempts=0,
                 reporter = reporter)

    frames = np.random.choice(range(num_frames), number_of_applications)
    particle = Particle(0)
    
    for i in tqdm.trange(number_of_applications):
        particle_state = OpenMMParticleState(positions = traj.xyz[frames[i]] * unit.nanometers, box_vectors = traj.unitcell_vectors[frames[i]]*unit.nanometers)
        particle.update_state(particle_state)
        try:
            _, _return_dict = ais_propagator.apply(particle_state, n_steps = steps_per_application, reset_integrator=True, apply_pdf_to_context=True)
        except Exception as e:
            print(e)


    return ais_propagator.state_works